<a href="https://colab.research.google.com/github/RGROVE1232/NE-snake-prediction/blob/main/snake_species_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Directory Operations and Random Number Generation
import os
import random

# Data Manipulation and Visualization
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
import pickle
import seaborn as sns

# Image Preprocessing and Callbacks
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from keras.callbacks import ReduceLROnPlateau

# Deep Learning Models/Layers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, Reshape, MaxPool2D,BatchNormalization
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Deep Learning Framework
import tensorflow as tf
from tensorflow import keras

In [ ]:
# Loading the model
mod = tf.keras.models.load_model("/content/drive/MyDrive/snake-dataset/model_v3.8.h5")

# Loading the history
with open('/content/drive/MyDrive/snake-dataset/model_history_v3.8.pkl', "rb") as file_pi:
    history = pickle.load(file_pi)

# List all data in history
print(history.keys())

testing_folder = '/content/drive/MyDrive/snake-dataset/testing'
image_files = os.listdir(testing_folder)

def preprocess_image(image_path):
    image = load_img(image_path, target_size=(224, 224))
    image = img_to_array(image)
    image = image / 255.0  # Normalize pixel values to [0, 1]
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy', 'lr'])


In [ ]:
species_info = {
    "Agkistrodon_contortrix": {
        "common_name": "Eastern Copperhead (VENOMOUS!)",
        "description_size": "Legless; no eyelids; lacks external ears; pit on each side of the head between the nostril and eye; vertical elliptical pupils; total length ~55.9-102.0 cm; banded pattern",
        "habitat": "hilly, wooded areas; rocky outcrops; open grasslands; agricultural fields",
        "diet": "Small mammals; amphibians; reptiles; birds; insects",
        "life_history": "Diurnal and nocturnal; active season, early April - late October; gives live birth; 3-7 young per female",
    },
    "Arizona_elegans": {
        "common_name": "Glossy snake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears; scales on the back are smooth; single anal scale; pupils slightly elliptical; relatively small head; total length ~63.5-178.0 cm",
        "habitat": "mixed to short-grass prairies; sand sage prairies; sandy soils; arid climate; sparse vegetation",
        "diet": "Lizards; small mammals",
        "life_history": "Nocturnal; active season, April - September; lays eggs; 3-23 eggs per female",
    },
    "Carphophis_vermis": {
        "common_name": "Western worm snake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears; scales on the back are smooth; small head; small eyes; sharp pointed tail; total length ~17.8-39.1 cm; purple coloration on top",
        "habitat": "moist woodlands; wooded hillsides; under rocks, bark, logs, and leaf litter",
        "diet": "Earthworms; insect larvae/pupae; very small snakes",
        "life_history": "Fossorial; active season, spring - mid fall; lays eggs; 1-6 eggs per female",
    },
    "Coluber_constrictor (NON-VENOMOUS)": {
        "common_name": "North American racer",
        "description_size": "Legless; no eyelids; lacks external ears; scales on the back are smooth; relatively large eyes; divided anal scale; total length ~51.0-190.0 cm",
        "habitat": "grasslands; agricultural fields; open woodlands; human dwellings",
        "diet": "Insects; reptiles; small mammals; birds; bird eggs; amphibians",
        "life_history": "Diurnal; active season, early April-September; lays eggs; 2-31 eggs per female",
    },
    "Coluber_flagellum (NON-VENOMOUS)": {
        "common_name": "Coachwhip",
        "description_size": "Legless; no eyelids; lacks external ears; scales on the back are smooth; relatively large eyes; divided anal scale; total length ~91.0-260.0 cm",
        "habitat": "grasslands; savannas; scrublands; sagebrush prairies; arid climates",
        "diet": "Small mammals; reptiles; small turtles; birds; bird eggs; insects",
        "life_history": "Diurnal, rarely nocturnal; active season, May-October; lays eggs; 1-2 clutches of 2-24 eggs per female",
    },
    "Crotalus_horridus (VENOMOUS!)": {
        "common_name": "Timber rattlesnake",
        "description_size": "Legless; no eyelids; lacks external ears; prominent ridge in the middle of each scale (strongly keeled); rattle presented on end of tail; vertical elliptical pupils; total length ~90.0-189.2 cm",
        "habitat": "deciduous woodlands; rocky outcrops",
        "diet": "Small mammals",
        "life_history": "Diurnal and nocturnal; active season, early April-late fall; gives live birth; 10-16 young per female",
    },
    "Crotalus_viridia": {
        "common_name": "Prairie rattlesnake (VENOMOUS!)",
        "description_size": "Legless; no eyelids; lacks external ears; prominent ridge in the middle of each scale (strongly keeled); rattle presented on end of tail; vertical elliptical pupils; total length ~84.0-144.8 cm",
        "habitat": "short and mixed-grass prairie; prairie dog towns; rocky mesa, canyons, and outcrops",
        "diet": "Small mammals; rarely birds; rarely lizards",
        "life_history": "Diurnal and nocturnal; active season, late March-mid November; gives live birth; 1-25 young per female",
    },
    "Diadophis_punctatus": {
        "common_name": "Ring-necked snake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears; scales on the back are smooth and glossy; total length ~25.0-41.9 cm; tail is brightly colored on the bottom; orange or red ring around its neck",
        "habitat": "open woodlands with rocky outcrops; woodland edges; debris piles in rural, suburban, and urban environments; grasslands",
        "diet": "Earthworms; small arthropods; small reptiles; small amphibians",
        "life_history": "Nocturnal; active season, late March - October; lays eggs; 1-2 clutches of 2-10 eggs per female",
    },
    "Heterodon_nasicus": {
        "common_name": "Plains hog-nosed snake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears; scales on the back have a prominent ridge in the middle (strongly keeled); divided anal scale; total length ~38.0 - 100.6 cm",
        "habitat": "dry short or mixed-grass prairie; sandy, loose soil; next to a body of water",
        "diet": "Toads and frogs; lizards; small mammals; reptile and bird eggs and hatchlings",
        "life_history": "Diurnal, rarely nocturnal; active season, April-October; lays eggs; 4-25 eggs per female",
    },
    "Lampropeltis_calligaster": {
        "common_name": "Prairie kingsnake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears; scales on the back are smooth; head has a 'V' shaped pattern, where the tip of the 'V' points towards the tail; single anal scale; total length ~76.0-142.7 cm",
        "habitat": "tallgrass prairie; savannas; woodland edges; sometimes agricultural fields",
        "diet": "Voles; small mammals; bird eggs; reptiles",
        "life_history": "Diurnal, sometimes nocturnal; active season, late March-late October; lays eggs; 8-12 eggs per female",
    },
    "Lampropeltis_gentilis": {
        "common_name": "Western milksnake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears; scales on the back are smooth; single anal scale; total length ~35.0-137.0 cm; tri-colored snake that bands consists of black, white, or yellow, and red or orange",
        "habitat": "grasslands; limestone or other rocky areas; woodland edges",
        "diet": "Small reptiles; small mammals; birds",
        "life_history": "Nocturnal, sometimes Diurnal; active season, early April-late October; lays eggs; 2-24 eggs per female",
    },
    "Lampropeltis_holbrooki": {
        "common_name": "Speckled kingsnake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    },
    "Nerodia_sipedon": {
        "common_name": "Common watersnake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 1",
        "diet": "Diet 1",
        "life_history": "Life History 1",
    },
    "Opheodrys_vernalis": {
        "common_name": "Smooth green snake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    },
    "Pantherophis_emoryi": {
        "common_name": "Great plains ratsnake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 1",
        "diet": "Diet 1",
        "life_history": "Life History 1",
    },
    "Pantherophis_obsoleta": {
        "common_name": "Western black ratsnake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    },
    "Pantherophis_ramspotti": {
        "common_name": "Western fox snake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    },
    "Pituophis_catenifer": {
        "common_name": "Bullsnake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    },
    "Regina_grahami": {
        "common_name": "Graham's crayfish snake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    },
    "Storeria_dekayi": {
        "common_name": "Dekay's Brownsnak (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    },
    "Storeria_occipitomaculata": {
        "common_name": "Red-bellied snake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    },
    "Tantilla_nigriceps": {
        "common_name": "Plains black-headed snake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    },
    "Thamnophis_proximus": {
        "common_name": "Western ribbonsnake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    },
    "Tropidoclonion_lineatum": {
        "common_name": "Lined snake (NON-VENOMOUS)",
        "description_size": "Legless; no eyelids; lacks external ears;",
        "habitat": "Habitat 2",
        "diet": "Diet 2",
        "life_history": "Life History 2",
    }
}